In [97]:
import yfinance as yf
import os 
import pandas as pd 
import numpy as np
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from datetime import datetime


In [98]:
"""url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp500_table = pd.read_html(url)[0]
sp500_tickers = sp500_table['Symbol'].tolist()"""
sp500_tickers = ["SPY"]
api_key = os.getenv("ALPACA_API_KEY")
secret_api_key = os.getenv("ALPACA_SECRET_KEY")
client = StockHistoricalDataClient(api_key, secret_api_key)


In [ ]:
total_data = []
folder = "data"
data_type = input("""Would you like the data tobe from Alpaca or YF?
                  Alpaca - 1
                  YF - 2
                  Note: Alpaca data requires env vars.""")
while data_type not in ["1", "2"]:
    print("input neither 1 or 2.")
    data_type = input("""Would you like the data tobe from Alpaca or YF?
                  Alpaca - 1
                  YF - 2
                  Note: Alpaca data requires env vars.""")
if data_type == "1":
    data_type = "Alpaca"
else:
    data_type = "YF"
for ticker in sp500_tickers:
    file = f"{ticker} {data_type} Data.parquet"
    file_path = os.path.join(folder, file)
    if os.path.exists(file_path):
        print(f"file for {ticker} already exists. Skipping...")
        continue
    try:
        if data_type == "YF":
            df = yf.download(ticker, period="max", progress=False)
        elif data_type == "Alpaca":
            max_timeframe = StockBarsRequest(
                    symbol_or_symbols=[ticker],
                    timeframe=TimeFrame.Day,
                    start=datetime(1999, 1, 1),
                    end=datetime.now(),
                    )
            max_timeframe = client.get_stock_bars(max_timeframe).df
            if max_timeframe.index.name != "timestamp":
                if "timestamp" in max_timeframe.columns:
                    max_timeframe.set_index("timestamp", inplace=True)
            if max_timeframe.empty:
                print(f"Downloaded data for {max_timeframe} was empty.")
                continue
            elif len(max_timeframe) < 200:
                print(f"{ticker} only has {len(max_timeframe)}")
                continue
            if isinstance(max_timeframe.columns, pd.MultiIndex):
                max_timeframe.columns = max_timeframe.columns.get_level_values(0)
            if isinstance(max_timeframe.index, pd.MultiIndex):
                max_timeframe.reset_index(inplace=True)
                max_timeframe.set_index("timestamp", inplace=True)

                max_timeframe.to_parquet(file)
        print(f"{ticker} done")
    except Exception as e:
        print(f"error, {e}")

                                     open    high     low     close  \
symbol timestamp                                                      
SPY    2016-01-04 05:00:00+00:00  200.490  201.03  198.59  201.0192   
       2016-01-05 05:00:00+00:00  201.405  201.90  200.05  201.3600   
       2016-01-06 05:00:00+00:00  198.330  200.06  197.60  198.8200   
       2016-01-07 05:00:00+00:00  195.330  197.44  193.59  194.0500   
       2016-01-08 05:00:00+00:00  195.190  195.85  191.58  191.9230   
...                                   ...     ...     ...       ...   
       2025-07-21 04:00:00+00:00  628.770  631.54  628.34  628.7700   
       2025-07-22 04:00:00+00:00  629.100  629.73  626.19  628.8600   
       2025-07-23 04:00:00+00:00  631.550  634.21  629.73  634.2100   
       2025-07-24 04:00:00+00:00  634.600  636.15  633.99  634.4200   
       2025-07-25 04:00:00+00:00  635.090  635.49  634.84  635.1650   

                                       volume  trade_count        vwap  
sym